In [1]:
##### import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random
import torch 

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
    
print(np.shape(uframes[0]))
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print("length: ",len(uframes))
for i in range(samples):
    data1.append(np.array(np.transpose(np.asarray(uframes[i % len(uframes)]),(2,0,1)),dtype=np.float32))
    data2.append(np.array(np.transpose(np.asarray(uframes[(i+1) % len(uframes)]),(2,0,1)),dtype=np.float32))

    

#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 6)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

epoch [4/200], loss:-0.9101
x:  tensor([[43.2569, 23.7910, 15.4159, -5.2288, 13.7124, 35.6697]],
       grad_fn=<AddmmBackward>)
y: tensor([[42.0363, 22.6163, 15.7179, -4.2939, 13.3839, 35.4136]],
       grad_fn=<AddmmBackward>)
b: tensor([[40.0236, 22.3985,  8.6696, -0.9512, 11.7391, 31.5031]],
       grad_fn=<AddmmBackward>)
epoch [5/200], loss:-1.3785
x:  tensor([[ 38.7607,  21.1320,  20.4806, -28.5197,  23.4375,   1.3371]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 38.8468,  20.9633,  20.4592, -28.8095,  23.8561,   1.9424]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 36.9773,  20.9558,  13.0849, -21.8618,  21.0104,   0.8077]],
       grad_fn=<AddmmBackward>)
epoch [6/200], loss:-1.3305
x:  tensor([[ 26.8728,   1.8034,   7.2132, -14.6885,  11.5596,  -9.4574]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 26.9036,   2.0725,   7.2248, -14.9850,  11.6208,  -9.6552]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 26.3367,   4.1189,   2.1770,  -8.1958,   8.9541, -11.1682]],
       gr

y: tensor([[ 30.2401, -28.9677, -10.1809,   9.7770,   5.3779,   8.8332]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 24.9188, -22.1531, -11.6391,   8.2579,   6.1653,   4.6191]],
       grad_fn=<AddmmBackward>)
epoch [29/200], loss:-1.5968
x:  tensor([[ 28.5509,   5.8746,  18.8231, -20.6583,  15.0288,   6.9221]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 28.4715,   5.2263,  19.3621, -19.8480,  15.4020,   7.0596]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 24.0078,   5.8688,  10.5675, -14.7297,  14.2983,   4.1840]],
       grad_fn=<AddmmBackward>)
epoch [30/200], loss:-1.4665
x:  tensor([[ 15.9119, -20.7440,   4.7133, -16.8579,   9.4242,   0.0942]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 15.8021, -21.2116,   5.0477, -16.1723,   9.6346,   0.0571]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 14.2294, -15.1690,  -0.7224,  -9.7212,   9.2359,  -1.0778]],
       grad_fn=<AddmmBackward>)
epoch [31/200], loss:-1.6641
x:  tensor([[  5.4106,  -9.5272,   7.3872,  -8.0058,   6.8963, 

y: tensor([[24.3208,  1.7239, 28.5469, -6.7696, -0.1282, 15.8424]],
       grad_fn=<AddmmBackward>)
b: tensor([[20.8953,  2.1777, 18.4752, -4.4660,  1.0532,  9.7150]],
       grad_fn=<AddmmBackward>)
epoch [53/200], loss:-1.5392
x:  tensor([[ 30.4672,  -9.4580,  32.2213, -12.6473,   3.6067,   6.1966]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 29.6907,  -9.5894,  32.3319, -13.4163,   3.7144,   5.0745]],
       grad_fn=<AddmmBackward>)
b: tensor([[23.0592, -6.2046, 21.0986, -7.4030,  4.2963,  2.1454]],
       grad_fn=<AddmmBackward>)
epoch [54/200], loss:-1.4321
x:  tensor([[ -1.0519,   9.8205,  19.3642, -41.2670,   1.1004,  11.7291]],
       grad_fn=<AddmmBackward>)
y: tensor([[ -0.6764,  10.2424,  19.1452, -42.2835,   0.8556,  12.3917]],
       grad_fn=<AddmmBackward>)
b: tensor([[  1.7237,   8.1238,  12.1089, -27.5473,   2.1346,   6.3937]],
       grad_fn=<AddmmBackward>)
epoch [55/200], loss:-1.4954
x:  tensor([[ 25.7171,  12.1687,  14.8255, -38.6574,   9.4402,  -9.8430]],
      

y: tensor([[ 18.7875, -14.4136,  16.9335, -48.0362,  -3.6254, -19.3463]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 15.2914,  -9.6369,   8.4001, -30.4344,  -1.3929, -15.5867]],
       grad_fn=<AddmmBackward>)
epoch [77/200], loss:-1.5877
x:  tensor([[  9.0693,  -3.3411,  11.4471, -12.9462,   2.4568,  -6.1416]],
       grad_fn=<AddmmBackward>)
y: tensor([[  8.9703,  -2.8665,  10.8040, -13.3372,   2.1972,  -7.0169]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 7.0361, -2.2454,  4.8392, -7.5691,  3.0105, -6.5004]],
       grad_fn=<AddmmBackward>)
epoch [78/200], loss:-1.3914
x:  tensor([[  2.0914,   3.6801, -14.6777, -28.9224,   1.4367, -13.9166]],
       grad_fn=<AddmmBackward>)
y: tensor([[  0.6949,   3.8778, -15.5311, -28.0081,   1.2859, -13.4009]],
       grad_fn=<AddmmBackward>)
b: tensor([[  3.2964,   2.6646, -12.9404, -17.8067,   1.7606,  -9.8927]],
       grad_fn=<AddmmBackward>)
epoch [79/200], loss:-1.6966
x:  tensor([[  9.3688,   6.1063,   0.6576, -24.9616,   1.4460, -14.24

y: tensor([[ 11.7158,   6.2120,  -2.1934, -12.7911,   3.1598,   0.0136]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 9.4348,  4.1802, -3.2845, -7.1786,  3.3082, -0.6845]],
       grad_fn=<AddmmBackward>)
epoch [101/200], loss:-1.6690
x:  tensor([[ 18.4598,  -4.1412,  -2.9501, -14.1607,  -5.5875, -12.1815]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 18.3668,  -3.6441,  -3.5639, -14.5362,  -5.8939, -13.0132]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 12.6987,  -2.5630,  -4.0219,  -8.8920,  -2.5576, -10.1585]],
       grad_fn=<AddmmBackward>)
epoch [102/200], loss:-1.6648
x:  tensor([[ 37.8918,   6.4470, -16.4436, -24.1650, -10.1629, -14.1064]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 37.8130,   6.9338, -17.0610, -24.5401, -10.4643, -14.9307]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 25.1747,   4.6242, -12.9340, -15.5513,  -5.8753, -11.4917]],
       grad_fn=<AddmmBackward>)
epoch [103/200], loss:-1.5742
x:  tensor([[ 32.8465,  11.8700,   3.3292, -22.8505,  -8.5542,  -5

b: tensor([[10.3248, -1.5719, 11.8067, -6.6266,  0.0486,  8.2531]],
       grad_fn=<AddmmBackward>)
epoch [125/200], loss:-1.4862
x:  tensor([[ -5.6689,   0.0723,  28.6269, -24.2154,   1.5346,  20.9880]],
       grad_fn=<AddmmBackward>)
y: tensor([[ -5.5267,   0.2352,  28.5672, -24.8814,   1.4634,  21.4719]],
       grad_fn=<AddmmBackward>)
b: tensor([[ -1.4207,   0.5759,  16.0516, -15.5995,   2.0563,  12.2004]],
       grad_fn=<AddmmBackward>)
epoch [126/200], loss:-1.7153
x:  tensor([[  1.6332,  -5.6752,  30.7417, -15.7940,  -5.1286,   5.5136]],
       grad_fn=<AddmmBackward>)
y: tensor([[  2.0115,  -5.3225,  30.6335, -16.8415,  -5.2978,   5.9971]],
       grad_fn=<AddmmBackward>)
b: tensor([[  2.7816,  -2.9388,  17.9100, -10.0119,  -1.7603,   2.3408]],
       grad_fn=<AddmmBackward>)
epoch [127/200], loss:-1.6036
x:  tensor([[  5.9403,   1.6354,  16.3473, -13.0192,  -4.9860,  12.4367]],
       grad_fn=<AddmmBackward>)
y: tensor([[  6.3714,   1.9914,  16.2865, -14.2167,  -5.1641,  12

epoch [149/200], loss:-1.6024
x:  tensor([[ -0.4360,  -3.3435,  -2.1373, -21.1423,  -1.6878,   6.0908]],
       grad_fn=<AddmmBackward>)
y: tensor([[ -0.0750,  -2.9928,  -2.2318, -22.2922,  -1.8994,   6.5539]],
       grad_fn=<AddmmBackward>)
b: tensor([[  1.8258,  -1.7809,  -2.4492, -13.7788,  -0.1164,   2.3658]],
       grad_fn=<AddmmBackward>)
epoch [150/200], loss:-1.3638
x:  tensor([[ 16.0851,   5.8951,  -4.0343, -29.7239,  -4.3289,   4.7633]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 15.9689,   5.4206,  -3.1901, -29.8697,  -4.0347,   4.4240]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 11.2744,   3.4787,  -3.4310, -17.6389,  -1.8134,   1.0981]],
       grad_fn=<AddmmBackward>)
epoch [151/200], loss:-1.4594
x:  tensor([[  2.8808,   8.4243,  13.2526, -18.8599,  -4.7237,   2.1977]],
       grad_fn=<AddmmBackward>)
y: tensor([[  2.8662,   7.9808,  14.1067, -18.9586,  -4.4701,   1.8505]],
       grad_fn=<AddmmBackward>)
b: tensor([[  2.9573,   5.0629,   7.1781, -11.6042,  -1.564

epoch [173/200], loss:-1.2199
x:  tensor([[ 5.9746,  7.3834, 22.0563, -3.9373, -4.7504, -2.5262]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 6.1827,  7.5603, 22.0538, -4.6121, -4.8525, -2.0875]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 5.3842,  5.1240, 12.2514, -2.9824, -1.5904, -2.8633]],
       grad_fn=<AddmmBackward>)
epoch [174/200], loss:-1.4983
x:  tensor([[ 12.7490,  -3.1948,  13.7606, -11.4855,  10.4198,  -8.3958]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 14.0867,  -3.2115,  13.8274, -10.7020,  10.3441,  -8.1672]],
       grad_fn=<AddmmBackward>)
b: tensor([[10.0237, -1.0333,  7.0731, -7.4607,  7.4189, -7.0124]],
       grad_fn=<AddmmBackward>)
epoch [175/200], loss:-1.5840
x:  tensor([[ 11.9947,   2.1035,  11.7582, -15.7465,  -7.1136,  -5.6942]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 12.3293,   2.4594,  11.6852, -16.8689,  -7.3235,  -5.3360]],
       grad_fn=<AddmmBackward>)
b: tensor([[  9.8493,   1.6761,   6.4059, -10.7804,  -3.4366,  -5.0600]],
       gr

epoch [198/200], loss:-1.5881
x:  tensor([[16.4587,  3.6416,  2.6552, -8.8006,  0.9330, -6.5585]],
       grad_fn=<AddmmBackward>)
y: tensor([[17.7566,  3.6272,  2.7119, -8.1201,  0.8534, -6.3551]],
       grad_fn=<AddmmBackward>)
b: tensor([[12.4744,  2.9199,  0.0803, -5.7160,  1.3806, -5.5934]],
       grad_fn=<AddmmBackward>)
epoch [199/200], loss:-1.4485
x:  tensor([[  7.3788,  -7.6843,  12.0052,  -8.6293, -10.9414,  -0.3579]],
       grad_fn=<AddmmBackward>)
y: tensor([[  6.1823,  -7.5640,  11.0877,  -7.8325, -10.9653,   0.6350]],
       grad_fn=<AddmmBackward>)
b: tensor([[ 6.3782, -3.7870,  5.8440, -5.6900, -5.5608, -0.6295]],
       grad_fn=<AddmmBackward>)
epoch [200/200], loss:-1.6253
x:  tensor([[ 16.0825,   3.7399,  -0.0653,  -0.1109, -11.0164,  -3.0978]],
       grad_fn=<AddmmBackward>)
y: tensor([[ 15.8724,   3.2458,   0.7621,  -0.1790, -10.7107,  -3.5247]],
       grad_fn=<AddmmBackward>)
b: tensor([[11.4153,  2.3901, -0.8193, -0.7659, -5.6721, -2.4328]],
       grad_fn=

In [3]:
data3=[]

class_data = []

for i in range(75):
    img = uframes[0]
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),0))



In [4]:
data3=[]

for i in range(75):
    img = uframes[1]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),2))
    
print(len(class_data))

150


In [5]:
data3=[]

for i in range(75):
    img = uframes[2]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),3))
    
print(len(class_data))

225


In [2]:
import random


p=0
class_data = []
while(p<len(uframes)):
    data3=[]
    print(p)
    for i in range(50):
        img = uframes[p]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        temp = model(x_1[i].reshape(1,3,100,100))
        #print(temp)
        class_data.append((list(temp.detach().numpy()[0]),p))
    p+=1
    print(len(class_data))

random.shuffle(class_data)
random.shuffle(class_data)

print("length ", len(class_data))

0
50
1
100
2
150
3
200
4
250
5
300
6
350
7
400
8
450
9
500
10
550
11
600
length  600


In [6]:
data3=[]

for i in range(75):
    img = uframes[3]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),1))
    
print(len(class_data))

300


In [18]:

cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

frame = Image.fromarray(uframes[0])


video = cv2.VideoWriter('video_test.avi', 0, 1, (720,1080))

for image in uframes:
    video.write(image)


for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
cv2.destroyAllWindows()
video.release()

a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from colorama import init 
from termcolor import colored 
import time
print(np.shape(uframes[0]))


#z  = np.array(np.transpose(np.asarray(z),(2,0,1)),dtype=np.float32)
# import some data to play with
print("Length of Class data: ", len(class_data))
# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
y3=0

n1 = "\x1b[31m\"Drone Stationary, Tank Firing\"\x1b[0m"
n2 = "\x1b[31m\"Drone Reloading, Tank Firing\"\x1b[0m"
n3 = "\x1b[31m\"Drone Firing, Tank Stationary\"\x1b[0m"
n4 = "\x1b[31m\"Drone Firing, Tank Reloading\"\x1b[0m"
n5 = "\x1b[31m\"Drone Firing, Tank Firing\"\x1b[0m"

names = ['Just Background','Only Tank moving', 'Only Drone Moving', 'Drone stationary, Tank Stationary', 'Drone Stationary, Tank Reloading' ,
         n1, 'Drone reloading, Tank Stationary', 'Drone reloading, Tank Reloading', n2 , n3,n4,n5]

print("Start")
for i in range(12):
    z = cv2.imread('Images/0d.png')
    z  = cv2.resize(z,(100,100))
    img = uframes[i]
    print(np.shape(img))
    #ms = Image.fromarray(z)
    #ms.show()
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))
    temp = temp.detach().numpy()
    print(temp)
    z = clf.predict(temp)
    print(names[z[0]])
    


ModuleNotFoundError: No module named 'termcolor'

In [54]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   2

In [55]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [53]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [2]:
img = np.transpose(data1[5],(1,2,0))
print(np.shape(np.transpose(data1[5],(1,2,0))))
cv2.imshow('test', img)

img = data2[0]
ms = Image.fromarray(img)
ms.show()

(100, 100, 3)


TypeError: Cannot handle this data type: (1, 1, 100), <f4